In [1]:
from langchain_community.llms import Cohere
from langchain.retrievers import ContextualCompressionRetriever, CohereRagRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.chat_models import ChatCohere
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass()

In [2]:
user_query = "Can my landlord evict me during the coronavirus pandemic?"

# Create cohere's chat model and embeddings objects
cohere_chat_model = ChatCohere()
cohere_embeddings = CohereEmbeddings() 

import pickle
with open("documents.pkl", "rb") as f:
    documents = pickle.load(f)

In [3]:
# Create a vector store from the documents
db = Chroma.from_documents(documents, cohere_embeddings)

# Create Cohere's reranker with the vector DB using Cohere's embeddings as the base retriever
cohere_rerank = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=cohere_rerank, 
    base_retriever=db.as_retriever()
)

compressed_docs = compression_retriever.get_relevant_documents(user_query)
# Print the relevant documents from using the embeddings and reranker
print(compressed_docs)

[Document(page_content='(5)\xa0\xa0If a tenant has given a landlord notice under subsection 53 (2) and subsection 54 (2) or (4) applies, the Board shall not issue an eviction order in a proceeding regarding termination of the tenancy until the landlord has compensated the tenant in accordance with subsection 54 (2) or (4), as applicable.  2006, c.\xa017, s.\xa083\xa0(5); 2020, c. 16, Sched. 4, s. 17 (2).\nRefusal for certain arrears of rent\n(6)\xa0\xa0Without restricting the generality of subsections (1) and (2), if a hearing is held in respect of an application under section 69 for an order evicting a tenant based on arrears of rent arising in whole or in part during the period beginning on March 17, 2020 and ending on the prescribed date, in determining whether to exercise its powers under subsection (1) the Board shall consider whether the landlord has attempted to negotiate an agreement with the tenant including terms of payment for the tenant’s arrears. 2020, c. 16, Sched. 4, s. 

In [4]:
# Create the cohere rag retriever using the chat model 
rag = CohereRagRetriever(llm=cohere_chat_model)

docs = rag.get_relevant_documents(
    user_query,
    source_documents=compressed_docs,
)

# Print the documents
for doc in docs[:-1]:
    print(doc.metadata)
    print("\n\n" + doc.page_content)
    print("\n\n" + "-" * 30 + "\n\n")
    
# Print the final generation 
answer = docs[-1].page_content
print(answer)
# Print the final citations 
citations = docs[-1].metadata['citations']
print(citations)

{'id': 'doc-0', 'snippet': '(5)\xa0\xa0If a tenant has given a landlord notice under subsection 53 (2) and subsection 54 (2) or (4) applies, the Board shall not issue an eviction order in a proceeding regarding termination of the tenancy until the landlord has compensated the tenant in accordance with subsection 54 (2) or (4), as applicable.  2006, c.\xa017, s.\xa083\xa0(5); 2020, c. 16, Sched. 4, s. 17 (2).\nRefusal for certain arrears of rent\n(6)\xa0\xa0Without restricting the generality of subsections (1) and (2), if a hearing is held in respect of an application under section 69 for an order evicting a tenant based on arrears of rent arising in whole or in part during the period beginning on March 17, 2020 and ending on the prescribed date, in determining whether to exercise its powers under subsection (1) the Board shall consider whether the landlord has attempted to negotiate an agreement with the tenant including terms of payment for the tenant’s arrears. 2020, c. 16, Sched. 4,